In [ ]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-
# vim:fenc=utf-8
#
# Copyright © 2022 Pi-Yueh Chuang <pychuang@gwu.edu>
#
# Distributed under terms of the BSD 3-Clause license.

"""Koopman analysis (dynamic mode decomposition).
"""

In [ ]:
import pathlib
import numpy
import scipy.signal
import scipy.fftpack
import scipy.linalg
import scipy.sparse.linalg
import h5py
from matplotlib import pyplot

In [ ]:
def read_snapshots(cachefile, chosen, fids, datadir):
    """Read snapshots.
    """

    # try to read aggregated data matrix
    try:
        with h5py.File(cachefile, "r") as dset:
            K = dset["K"][...]
            xm = dset["xm"][...]
            times = dset["times"][...]
        print(f"Done reading aggregated data matrix from {cachefile.name}")
    except FileNotFoundError:
        times = numpy.zeros(len(fids), dtype=float)  # to store time values
        K = {
            field: numpy.zeros((numpy.count_nonzero(idxs), len(fids)-1))
            for field, idxs in chosen.items()
        }  # snapshot matrix
        xm = {
            field: numpy.zeros(numpy.count_nonzero(idxs))
            for field, idxs in chosen.items()
        }  # the last snapshot

        # store data to K
        for i, fid in enumerate(fids[:-1]):
            with h5py.File(datadir.joinpath(f"{fid:07d}.h5"), "r") as dset:
                K["u"][:, i] = dset["u"][chosen["u"]].flatten()
                K["v"][:, i] = dset["v"][chosen["v"]].flatten()
                K["p"][:, i] = dset["p"][chosen["p"]].flatten()
                times[i] = float(dset["p"].attrs["time"])
            print(f"Done reading solution at t={times[i]} from {fid:07d}.h5")

        # the last snapshot
        with h5py.File(datadir.joinpath(f"{fids[-1]:07d}.h5"), "r") as dset:
            xm["u"][...] = dset["u"][chosen["u"]].flatten()
            xm["v"][...] = dset["v"][chosen["v"]].flatten()
            xm["p"][...] = dset["p"][chosen["p"]].flatten()
            times[-1] = float(dset["p"].attrs["time"])
        print(f"Done reading solution at t={times[-1]} from {fids[-1]:07d}.h5")

        # concatenate
        K = numpy.concatenate((K["u"], K["v"], K["p"]), axis=0)
        xm = numpy.concatenate((xm["u"], xm["v"], xm["p"]), axis=0)

        # save
        with h5py.File(cachefile, "w") as dset:
            dset.create_dataset("K", shape=K.shape, dtype=K.dtype, data=K, compression=True)
            dset.create_dataset("xm", shape=xm.shape, dtype=xm.dtype, data=xm, compression=True)
            dset.create_dataset("times", shape=times.shape, dtype=times.dtype, data=times, compression=True)

    return K, xm, times

In [ ]:
def get_companion_matrix(cachefile, K, xm):
    """Get the companion matrix.
    """

    # try to read economic-QR data from cache
    try:
        with h5py.File(cachefile, "r") as dset:
            Q = dset["Q"][...]
            R = dset["R"][...]
        print(f"Done reading cached economic-QR data from {cachefile.name}")
    except KeyError:
        print("No economic-QR cache available. Re-calculate economic-QR.")
        Q, R = scipy.linalg.qr(K, mode="economic")
        print("Done")

        # save
        with h5py.File(cachefile, "r+") as dset:
            dset.create_dataset("Q", shape=Q.shape, dtype=Q.dtype, data=Q, compression=True)
            dset.create_dataset("R", shape=R.shape, dtype=R.dtype, data=R, compression=True)

    # try to read economic-QR data from cache
    try:
        with h5py.File(cachefile, "r") as dset:
            c = dset["c"][...]
        print(f"Done reading cached c vector from {cachefile.name}")
    except KeyError:
        print("No cached c vector. Re-calculate it.")
        c = scipy.linalg.solve(R, Q.T.dot(xm))
        print("Done")

        # save
        with h5py.File(cachefile, "r+") as dset:
            dset.create_dataset("c", shape=c.shape, dtype=c.dtype, data=c, compression=True)

    # residual check
    assert numpy.allclose(K.T.dot(xm-K.dot(c)), 0, 1e-8, 1e-8)

    # construct C matrix
    print("Constructing the companion matrix C")
    C = numpy.zeros((K.shape[1], K.shape[1]))
    C[range(1, K.shape[1]), range(0, K.shape[1]-1)] = 1.0
    C[:, -1] = c

    return C

In [ ]:
def get_modes(datadir, outdir, lims, radius, fids):
    """Get Koopman modes and eigenvalues.
    """

    # coordinates of the nodes
    with h5py.File(datadir.joinpath("grid.h5"), "r") as dset:
        coords = {
            "u": numpy.meshgrid(dset["u"]["x"], dset["u"]["y"], indexing="xy"),
            "v": numpy.meshgrid(dset["v"]["x"], dset["v"]["y"], indexing="xy"),
            "p": numpy.meshgrid(dset["p"]["x"], dset["p"]["y"], indexing="xy"),
        }

    # a boolean array indicating cells inside the region of interest and outside the cylinder
    chosen = {
        field: (
            (coords[field][0] >= lims[0]) & (coords[field][0] <= lims[1]) &
            (coords[field][1] >= lims[2]) & (coords[field][1] <= lims[3]) &
            (coords[field][0]**2 + coords[field][1]**2 >= radius**2)
        ) for field in coords.keys()
    }

    # singular value decomposition
    fname = outdir.joinpath("svd_uvp.nosync")

    # eigenvalues, eigenvectors, and diagonalization
    try:
        with h5py.File(fname, "r") as dset:
            times = dset["times"][...]
            sigma = dset["sigma"][...]
            V = dset["V"][...]
        print("Done reading eigenvalues and eigenvectors.")
    except (KeyError, FileNotFoundError):

        # try to read aggregated data matrix
        K, xm, times = read_snapshots(fname, chosen, fids, datadir)

        # get the companion matrix
        C = get_companion_matrix(fname, K, xm)

        print("Re-calculating eigenvalues and eigenvectors.")
        sigma, T = scipy.linalg.eig(C)
        V = K.dot(T)
        print("Done")

        # save
        with h5py.File(fname, "r+") as dset:
            dset.create_dataset("sigma", shape=sigma.shape, dtype=sigma.dtype, data=sigma, compression=True)
            dset.create_dataset("V", shape=V.shape, dtype=V.dtype, data=V, compression=True)
            dset.create_dataset("T", shape=T.shape, dtype=T.dtype, data=T, compression=True)

    return chosen, sigma, V, coords

In [ ]:
def plot_eigenvalue_dist(sigmas, energy, figdir):
    """Create plots.
    """

    # assuming the original order in decreasing, change to increasing order
    sigmas = sigmas[-1::-1]
    energy = energy[-1::-1]

    # find the index of non-time-averaged modes
    idx = numpy.where(abs(sigmas.imag) > 1e-10)[0]

    pyplot.figure()

    # plot modes other than mode 0
    pyplot.scatter(
        sigmas[idx].real, sigmas[idx].imag, s=40, c=energy[idx],
        edgecolors="k", linewidth=0.5, cmap="inferno_r", alpha=0.8,
    )
    pyplot.colorbar(label="Strength of corresponding Koopman mode")

    # plot mode 0
    pyplot.scatter(
        1.0, 0.0, s=80, c="crimson", marker="X", edgecolors="k", linewidth=0.5,
        label=r"Steady mode (Strength$\approx 10.3$)"
    )

    pyplot.xlim(-1.1, 1.1)
    pyplot.ylim(-1.1, 1.1)
    pyplot.gca().axis('equal')

    pyplot.xlabel("Real part of the eigenvalue")
    pyplot.ylabel("Imaginary part of the eigenvalue")

    pyplot.legend(loc="center")

    pyplot.title("Koopman eigenvalues")
    pyplot.savefig(figdir.joinpath("koopman_petibm_eigenvalues_lumped.png"))
    pyplot.close()

In [ ]:
def plot_mode_strengths(freqs, energy, figdir):
    """Plot bar plot of energy v.s. frequencies (excluding time-averaged mode).

    Notes
    -----
    Assuming 1) freqs are non-negative and 2) freqs and energy are sorted in an decreasing order.
    """

    # find non-time-averaged modes
    idx = numpy.where((freqs > 0.) & (freqs <= 5.))[0]
    freqs = freqs[idx]
    energy = energy[idx]

    # sort with decreasing energy
    idx = numpy.argsort(energy)
    freqs = freqs[idx][-1::-1]
    energy = energy[idx][-1::-1]

    # get colors
    cmap = pyplot.get_cmap("inferno_r")
    normalizer = pyplot.Normalize(vmin=energy.min(), vmax=energy.max())

    pyplot.figure()
    pyplot.bar(freqs, energy, width=0.15, color=cmap(normalizer(energy)))
    pyplot.annotate(
        rf"$St={freqs[0]:.3f}$", (freqs[0], energy[0]), (freqs[0]+0.5, energy[0]),
        arrowprops={"arrowstyle": "<-", "relpos": (0, 0.5), "color": "dimgray"}
    )
    pyplot.annotate(
        rf"$St={freqs[1]:.3f}$", (freqs[1], energy[1]), (freqs[1]+0.5, energy[1]+0.2),
        arrowprops={"arrowstyle": "<-", "relpos": (0, 0.5), "color": "dimgray"}
    )
    pyplot.xlim(0., 5.)
    pyplot.xlabel(r"Strouhal number ($St$)")
    pyplot.ylabel("Mode strength")
    pyplot.title("Strength of Koopman modes")
    pyplot.savefig(figdir.joinpath("koopman_petibm_mode_strength.png"))
    pyplot.close()

In [ ]:
def plot_modes(modes, freqs, energy, chosen, coords, figdir, k=5):
    """Plot contours for modes.
    """

    names = {"u": r"$u$ velocity", "v": r"$v$ velocity", "p": "Pressure"}

    for field in ["u", "v", "p"]:
        for i in range(energy.size):
            data = numpy.zeros_like(coords[field][0])
            data[chosen[field]] = modes[field][:, i].real
            data = numpy.ma.array(data, mask=numpy.invert(chosen[field]))

            vmean = data.mean()
            vstd = data.std()
            lvls = 128  # numpy.linspace(vmean-3*vstd, vmean+3*vstd, 64)

            pyplot.figure(figsize=(6.5, 3.5))
            pyplot.contourf(
                coords[field][0], coords[field][1], data,
                levels=lvls, cmap="turbo"
            )
            pyplot.gca().add_artist(pyplot.Circle((0., 0.), 0.5, color="w"))
            pyplot.gca().set_xlim(-3, 14)
            pyplot.gca().set_ylim(-3, 3)
            pyplot.gca().set_aspect("equal", "box")
            pyplot.gca().set_xlabel("x")
            pyplot.gca().set_ylabel("y")
            pyplot.colorbar(orientation="horizontal", aspect=50, label=names[field])
            pyplot.title(rf"Koopman mode {i}, field: {field} ($St={freqs[i]:.3f}$, strength$\approx{energy[i]:.2f}$)")
            pyplot.savefig(figdir.joinpath(f"koopman_petibm_mode_{field}_{i:02d}.png"))
            pyplot.close()

            print(f"Done plotting the contour of mode {i}, field {field}")

            if i >= 5:
                break


In [ ]:
# directories and paths
_projdir = pathlib.Path.cwd().parents[2]
_datadir = _projdir.joinpath("petibm", "cylinder-2d-re200-koopman.nosync", "output")
_figdir = _projdir.joinpath("modulus", "cylinder-2d-re200", "postprocessing", "_figs", "koopman.nosync")
_figdir.mkdir(exist_ok=True)

# unified figure style
pyplot.style.use(_projdir.joinpath("resources", "figstyle"))

# some parameters
_dt = 0.005
_nsteps = 10
_fids = numpy.arange(25000, 28001, _nsteps, dtype=int)
_radius = 0.5
_lims = (-3, 14, -3, 3)

_chosen, _sigmas, _modes, _coords = get_modes(_datadir, _figdir, _lims, _radius, _fids)

In [ ]:
# energy norm
_energy = numpy.linalg.norm(numpy.absolute(_modes), axis=0)

# sort by energy norm (in a decreasing order)
_idxs = numpy.argsort(_energy)[::-1]
_sigmas = _sigmas[_idxs]
_modes = _modes[:, _idxs]
_energy = _energy[_idxs]

# plot using both positive and negative frequencies
plot_eigenvalue_dist(_sigmas, _energy, _figdir)

In [ ]:
# calculate growth rates and frequencies
_lambdas = numpy.log(_sigmas).real  # growth rates
_omegas = numpy.log(_sigmas).imag / (_dt * _nsteps)  # circular frequencies
_freqs = _omegas / (2. * numpy.pi)  # frequencies

# only retain non-negative frequencies
_pos = (_freqs >= 0.)
_sigmas = _sigmas[_pos]
_modes = _modes[:, _pos]
_energy = _energy[_pos]
_lambdas = _lambdas[_pos]
_omegas = _omegas[_pos]
_freqs = _freqs[_pos]

# plot strength distribution
plot_mode_strengths(_freqs, _energy, _figdir)

In [ ]:
# split eigenvectors to each field
_modes = numpy.split(
    _modes,
    numpy.array([_chosen["u"].sum(), _chosen["u"].sum()+_chosen["v"].sum()]),
    axis=0
)
_modes = {"u": _modes[0], "v": _modes[1], "p": _modes[2]}

In [ ]:
plot_modes(_modes, _freqs, _energy, _chosen, _coords, _figdir, k=5)